In [135]:
! pip install -q --upgrade google-generativeai langchain-google-genai python-dotenv

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
import google.generativeai as genai
import os
genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))

In [6]:
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

In [7]:
import json
with open("a song of ice and fire.json", "r", encoding="utf-8") as file:
    data_dict = json.load(file)

In [8]:

docs = ''.join(list(data_dict.values()))
# data = sum(docs.values())
# data

In [9]:
with open("background.txt", "r", encoding="utf-8") as file:
    docs = file.read()

In [1]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

# load the document and split it into chunks
loader = TextLoader("background.txt")
documents = loader.load()

# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# load it into Chroma
db = Chroma.from_documents(docs, embedding_function)

# query it
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)

# print results
print(docs[0].page_content)

Created a chunk of size 1048, which is longer than the specified 1000


The second part of the story takes place in the far north of Westeros, where an 8,000-year-old wall of ice, simply called "the Wall", defends the Seven Kingdoms from supernatural creatures known as the Others. The Wall's sentinels, the Sworn Brotherhood of the Night's Watch, also protect the realm from incursions by the "wildlings" or "Free Folk", a number of human tribes living on the north side of the Wall. The Night's Watch story is told primarily through the point of view of Jon Snow, Lord Eddard Stark's bastard son.[11] Jon follows the footsteps of his uncle Benjen Stark and joins the Watch at a young age, rising quickly through the ranks. He eventually becomes Lord Commander of the Night's Watch. In the third volume, A Storm of Swords, the Night's Watch storyline becomes increasingly entangled with the War of the Five Kings.


In [6]:
query = "king"
docs = db.similarity_search(query)

# print results
print(docs[0].page_content)

Plot synopsis
Further information: List of A Song of Ice and Fire characters and World of A Song of Ice and Fire
A Song of Ice and Fire takes place in a fictional world in which seasons last for years and end unpredictably. Nearly three centuries before the events of the first novel, the Seven Kingdoms of Westeros were united under the Targaryen dynasty, establishing military supremacy through their control of dragons. The Targaryens ruled for three hundred years, continuing beyond the extinction of the dragons. Their dynasty eventually ended with a rebellion led by Lord Robert Baratheon, in which Aerys "the Mad King" Targaryen was killed and Robert proclaimed king of the Seven Kingdoms. At the beginning of A Game of Thrones, 15 years have passed since Robert's rebellion, with a nine-year-long summer coming to an end.


In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
texts = text_splitter.create_documents([docs])
len(texts)
texts[0].page_content

'A Song of Ice and Fire is a series of epic fantasy novels by the American novelist and screenwriter George R. R. Martin. He began writing the first volume, A Game of Thrones, in 1991, publishing it in 1996. Martin originally envisioned the series as a trilogy but has released five out of a planned seven volumes. The fifth and most recent entry in the series, A Dance with Dragons, was published in 2011. Martin continues to write the sixth novel, titled The Winds of Winter.[3] A seventh novel, A'

In [11]:
simple_texts = [i.page_content for i in texts]
# test_text = "丹妮莉丝·坦格利安（Daenerys Targaryen），又称丹妮或者“风暴降生”丹妮莉丝。她是“疯王”伊里斯·坦格利安二世之女，雷加·坦格利安王子的妹妹，也是古老的坦格利安王朝最后的传人。她在童年时与哥哥韦赛里斯·坦格利安一直被作为他人的政治筹码。韦赛里斯为了获取多斯拉克人的大军，将丹妮嫁给卓戈卡奥。自此丹妮莉丝逐渐成长为一个令人敬畏的女王和解放者。因为她从龙蛋中孵化出了三只幼龙，又被成为“龙之母”。丹妮莉丝是小说中一个主要的POV人物。在电视剧权力的游戏中，丹妮莉丝由演员Emilia"
simple_texts

['A Song of Ice and Fire is a series of epic fantasy novels by the American novelist and screenwriter George R. R. Martin. He began writing the first volume, A Game of Thrones, in 1991, publishing it in 1996. Martin originally envisioned the series as a trilogy but has released five out of a planned seven volumes. The fifth and most recent entry in the series, A Dance with Dragons, was published in 2011. Martin continues to write the sixth novel, titled The Winds of Winter.[3] A seventh novel, A',
 'Dream of Spring, is planned to follow.[4]',
 'A Song of Ice and Fire depicts a violent world dominated by political realism. What little supernatural power that remains is confined to the margins of the known world. Moral ambiguity pervades the books and their stories continually raise questions concerning loyalty, pride, human sexuality, piety, and the morality of violence.',
 'The novels are set on the fictional continents of Westeros and Essos. The story unfolds through a rotating set of

In [12]:
query_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
doc_embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001", task_type="retrieval_document"
)
db = Chroma.from_documents(texts, query_embeddings)
#test_text = "丹妮莉丝·坦格利安（Daenerys Targaryen），又称丹妮或者“风暴降生”丹妮莉丝。她是“疯王”伊里斯·坦格利安二世之女，雷加·坦格利安王子的妹妹，也是古老的坦格利安王朝最后的传人。她在童年时与哥哥韦赛里斯·坦格利安一直被作为他人的政治筹码。韦赛里斯为了获取多斯拉克人的大军，将丹妮嫁给卓戈卡奥。自此丹妮莉丝逐渐成长为一个令人敬畏的女王和解放者。因为她从龙蛋中孵化出了三只幼龙，又被成为“龙之母”。丹妮莉丝是小说中一个主要的POV人物。在电视剧权力的游戏中，丹妮莉丝由演员Emilia"
#query_embeddings.embed_documents(texts)


In [22]:

query = "asd"
query_embeddings.embed_query(query)
db.as_retriever(query)

TypeError: VectorStore.as_retriever() takes 1 positional argument but 2 were given

In [14]:
def make_qa_chain():
    llm = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True)
    return RetrievalQA.from_chain_type(
        llm,
        retriever=db.as_retriever(),
        return_source_documents=True
    )
qa_chain = make_qa_chain()

In [17]:
q = "tell me about the stark family"
result = qa_chain({"query": q})
print(result["result"],"\nsources:",)
for idx, elt in enumerate(result["source_documents"]):
    print(elt.page_content)

I cannot answer this question since the provided text does not contain any information about the Stark family. 
sources:
Plot synopsis
Further information: List of A Song of Ice and Fire characters and World of A Song of Ice and Fire
progress by the middle of the second book, A Clash of Kings.
A Song of Ice and Fire depicts a violent world dominated by political realism. What little supernatural power that remains is confined to the margins of the known world. Moral ambiguity pervades the books and their stories continually raise questions concerning loyalty, pride, human sexuality, piety, and the morality of violence.
The principal story chronicles the power struggle for the Iron Throne among the great Houses of Westeros following the death of King Robert in A Game of Thrones. Robert's heir apparent, the 13-year-old Joffrey, is immediately proclaimed king through the machinations of his mother, Queen Cersei Lannister. When Lord Eddard "Ned" Stark, Robert's closest friend and chief adv